In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Replace 'your_file.csv' with the name of your CSV file
file_path = '/content/drive/My Drive/datamining/your_file.csv'



Mounted at /content/drive


In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
import random

# Load data
train_df = pd.read_csv('/content/drive/My Drive/datamining/train.csv')
test_df = pd.read_csv('/content/drive/My Drive/datamining/test.csv')
test_label_df = pd.read_csv('/content/drive/My Drive/datamining/test_label.csv')

# Normalize the features
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_df)
test_scaled = scaler.transform(test_df)

# Define a dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Create datasets
train_dataset = TimeSeriesDataset(train_scaled)
test_dataset = TimeSeriesDataset(test_scaled)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [8]:
class TransformerAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        print(input_dim)
        super(TransformerAutoencoder, self).__init__()
        self.encoder = nn.TransformerEncoderLayer(d_model=input_dim, nhead=4, dim_feedforward=hidden_dim, batch_first=True)
        self.decoder = nn.TransformerEncoderLayer(d_model=input_dim, nhead=4, dim_feedforward=hidden_dim, batch_first=True)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )

    def forward(self, x):
        return self.model(x)

# Initialize models
input_dim = train_scaled.shape[1]
hidden_dim = 128
autoencoder = TransformerAutoencoder(input_dim, hidden_dim, 3)
discriminator = Discriminator(input_dim)
generator = Generator(input_dim, input_dim)


26


AssertionError: embed_dim must be divisible by num_heads

In [9]:
class TransformerAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(TransformerAutoencoder, self).__init__()
        # Adjust the number of heads to fit your specific input_dim
        num_heads = 4 if input_dim % 4 == 0 else 2 if input_dim % 2 == 0 else 1
        self.encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True), num_layers=num_layers)
        self.decoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True), num_layers=num_layers)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Assuming your input_dim here. Replace it with the actual dimension if different.
input_dim = train_scaled.shape[1]
hidden_dim = 128
autoencoder = TransformerAutoencoder(input_dim, hidden_dim, num_layers=3)
discriminator = Discriminator(input_dim)
generator = Generator(input_dim, input_dim)

# Continue with the rest of the setup and training as previously described


In [10]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        self.eps = 1e-9

    def forward(self, pos, neg):
        # Distance between the positive pairs (should be low)
        pos_dist = torch.sqrt(torch.sum((pos[0] - pos[1]) ** 2, dim=1) + self.eps)
        # Distance between the negative pairs (should be high)
        neg_dist = torch.sqrt(torch.sum((neg[0] - neg[1]) ** 2, dim=1) + self.eps)

        # Contrastive loss calculation
        loss = torch.mean(pos_dist + torch.clamp(self.margin - neg_dist, min=0))
        return loss


In [11]:
def augment_data(data, mask_rate=0.2):
    mask = np.random.geometric(p=mask_rate, size=data.shape) > 1
    return data * mask


AttributeError: 'numpy.ndarray' object has no attribute 'to'

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:266: UserWarning: Error detected in MseLossBackward0. Traceback of forward call that caused the error:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  File "/usr/

RuntimeError: Function 'MseLossBackward0' returned nan values in its 0th output.

/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:266: UserWarning: Error detected in MseLossBackward0. Traceback of forward call that caused the error:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  File "/usr/

RuntimeError: Function 'MseLossBackward0' returned nan values in its 0th output.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Assuming definitions of your models, optimizers, and data loaders

# Enable anomaly detection
torch.autograd.set_detect_anomaly(True)

# Training loop
epochs = 50
for epoch in range(epochs):
    for data in train_loader:
        real_data = data.to(device)
        augmented_data_np = augment_data(real_data.cpu().numpy())
        augmented_data = torch.from_numpy(augmented_data_np).to(device)

        # --- Train Autoencoder ---
        optimizer_ae.zero_grad()

        # Convert reconstructed_data to Float
        reconstructed_data = autoencoder(real_data.float())

        # Remove NaN values from input and output
        mask = ~torch.isnan(reconstructed_data) & ~torch.isnan(real_data)
        reconstructed_data = reconstructed_data[mask]
        real_data = real_data[mask]

        if reconstructed_data.numel() == 0:
            print("All samples in the batch contain NaN values. Skipping this batch.")
            continue

        loss_ae = criterion_ae(reconstructed_data.float(), real_data.float())
        loss_ae.backward()
        optimizer_ae.step()

        # --- Train Discriminator ---
        optimizer_d.zero_grad()
        real_labels = torch.ones(real_data.size(0), 1).to(device)
        fake_labels = torch.zeros(real_data.size(0), 1).to(device)

        real_output = discriminator(real_data.float())

        # Remove NaN values from discriminator output and labels
        mask = ~torch.isnan(real_output) & ~torch.isnan(real_labels)
        real_output = real_output[mask]
        real_labels = real_labels[mask]

        loss_d_real = criterion_gan_d(real_output, real_labels)

        fake_data = generator(augmented_data.float()).detach()
        fake_output = discriminator(fake_data)

        # Remove NaN values from discriminator output and labels
        mask = ~torch.isnan(fake_output) & ~torch.isnan(fake_labels)
        fake_output = fake_output[mask]
        fake_labels = fake_labels[mask]

        loss_d_fake = criterion_gan_d(fake_output, fake_labels)

        loss_d = (loss_d_real + loss_d_fake) / 2
        loss_d.backward()
        optimizer_d.step()

        # --- Train Generator ---
        optimizer_g.zero_grad()
        fake_data = generator(augmented_data)
        fake_output = discriminator(fake_data)
        loss_g = criterion_gan_g(fake_output, real_labels)
        loss_g.backward()
        optimizer_g.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss AE: {loss_ae.item()}, Loss D: {loss_d.item()}, Loss G: {loss_g.item()}')


All samples in the batch contain NaN values. Skipping this batch.
All samples in the batch contain NaN values. Skipping this batch.
All samples in the batch contain NaN values. Skipping this batch.
All samples in the batch contain NaN values. Skipping this batch.
All samples in the batch contain NaN values. Skipping this batch.
All samples in the batch contain NaN values. Skipping this batch.
All samples in the batch contain NaN values. Skipping this batch.
All samples in the batch contain NaN values. Skipping this batch.
All samples in the batch contain NaN values. Skipping this batch.
All samples in the batch contain NaN values. Skipping this batch.
All samples in the batch contain NaN values. Skipping this batch.
All samples in the batch contain NaN values. Skipping this batch.
All samples in the batch contain NaN values. Skipping this batch.
All samples in the batch contain NaN values. Skipping this batch.
All samples in the batch contain NaN values. Skipping this batch.
All sample